In [217]:
import pandas as pd
import pandas_datareader as web
import requests
import numpy as np

In [246]:
fp_out = r"C:\Users\micha\OneDrive\Documents\data\lstm_data.csv"

# Famma French

In [219]:
from pandas_datareader._utils import RemoteDataError

def get_ff_factors(START_DATE):
    try:
        df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench',
                                        start=START_DATE)[0]
        df_five_factor = df_five_factor.div(100)
        df_five_factor.index = df_five_factor.index.strftime('%Y-%m-%d')
        df_five_factor.index.name = 'Date'
        df_five_factor.columns = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
        df_five_factor.index = pd.to_datetime(df_five_factor.index)

        momentum_factor = web.DataReader('F-F_Momentum_Factor_daily', 'famafrench', START_DATE  )[0]

        combined_factors = df_five_factor.join(momentum_factor)

        return combined_factors
    except RemoteDataError as e:
        print(f"Failed to fetch data: {e}")
        return None

# Example usage
ff = get_ff_factors(START_DATE='1990-01-01')

In [220]:
ff = ff.shift(1).dropna()

In [221]:
ff.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1990-01-03,0.0144,-0.0067,-0.0007,0.0019,-0.0044,0.00026,-1.09
1990-01-04,-0.0006,0.0072,-0.0026,0.0014,-0.0043,0.00026,-0.31
1990-01-05,-0.0071,0.0044,-0.0022,-0.0006,-0.0003,0.00026,-0.38
1990-01-08,-0.0085,0.0074,-0.0021,-0.0001,-0.0012,0.00026,-0.33
1990-01-09,0.0030,-0.0040,-0.0026,0.0006,0.0031,0.00026,0.18


# Fred

In [222]:
import pandas as pd
import requests
from pathlib import Path
import sys, os
import json

frequency_mapping = {
        "GDP": "Q",  # Quarterly
        "CPIAUCSL": "M",  # Monthly
        "CPILFESL": "M",  # Monthly; Core CPI
        "PCE": "M",  # Monthly; Personal Consumption Expenditures,
        "UNRATE": "M",  # Monthly
        "JTSJOL":"M",
        "SP500": "D",  # Daily
        "DGS10": "D",  # Daily
        "DGS5": "D",  # Daily
        "DGS1MO": "D",  # Daily
        "DGS3MO": "D",  # Daily
        "DGS6MO": "D",  # Daily
        "DGS1": "D",  # Daily
        "DGS2": "D",  # Daily
        "DGS3": "D",  # Daily
        "DGS7": "D",  # Daily
        "DGS20": "D",  # Daily
        "DGS30": "D",  # Daily
        "DCOILWTICO": "D",  # Daily; WTI Crude Oil Price
        "DCOILBRENTEU": "D",  # Daily; Brent
        "DAAA": "D",  # Daily
        "DBAA": "D",  # Daily
        "AAA": "D",  # Daily
        "BAA": "D",  # Daily
        "BAMLH0A0HYM2": "D",  # Daily
        "T10Y2Y": "D",  # Daily
        "T10Y3M": "D",  # Daily
        "T10YFF": "D",  # Daily
        "T10YIE": "D",  # Daily
        "T5YFFM": "D",  # Daily
        "CBBTCUSD": "D",  # Daily
        "EXPINF2YR": "M",  # Daily
    }

def get_matrix(ids=frequency_mapping.keys()):
    """ Retrieve multiple Fred data series' and return a dataframe. """
    parent = Path('').resolve()#.parent
    SECRET_FP = os.path.join(parent, "secrets.json")
    with open(SECRET_FP, 'r') as file:
        secrets = json.load(file)
    API_KEY = secrets['fred_api_key']

    merged_data = None

    for series_id in ids:
        params = {
            "series_id": series_id,
            "api_key": 'ae9eac413fee6bc7cbe0747b78d0b32c',
            "file_type": "json",
        }
        FRED_URL = "https://api.stlouisfed.org/fred/series/observations"
        response = requests.get(FRED_URL, params=params)

        if response.status_code == 200:
            data = response.json()
            observations = data["observations"]
            df = pd.DataFrame(observations)
            df["value"] = pd.to_numeric(df["value"], errors="coerce")
            df = df[["date", "value"]]
            df.rename(columns={"value": series_id}, inplace=True)
            df["date"] = pd.to_datetime(df["date"])
            df.set_index("date", inplace=True)

            # Adjust for look-ahead bias
            frequency = frequency_mapping.get(series_id, "D")  # Default to daily if not specified
            if frequency == "QE":
                df = df.shift(1, freq='QE')
            elif frequency == "ME":
                df = df.shift(1, freq='ME')
            else:
                df = df.shift(1)

            if merged_data is None:
                merged_data = df
            else:
                merged_data = merged_data.merge(
                    df, how="outer", left_index=True, right_index=True
                )

        else:
            print(f"Failed to fetch data for {series_id}. Status code: {response.status_code}")
            pass

    merged_data.index = pd.to_datetime(merged_data.index)

    return merged_data

macro = get_matrix()

In [223]:
macro = macro.ffill().dropna()

In [224]:
macro.head()

,GDP,CPIAUCSL,CPILFESL,PCE,UNRATE,JTSJOL,SP500,DGS10,DGS5,DGS1MO,...,AAA,BAA,BAMLH0A0HYM2,T10Y2Y,T10Y3M,T10YFF,T10YIE,T5YFFM,CBBTCUSD,EXPINF2YR
date,,,,,,,,,,,,,,,,,,,,,
2015-02-20,17912.079,234.747,239.811,12066.7,5.7,5344.0,2097.45,2.11,1.58,0.02,...,3.46,4.45,4.58,1.44,2.09,1.99,1.74,1.26,240.90,1.25126
2015-02-21,17912.079,234.747,239.811,12066.7,5.7,5344.0,2097.45,2.11,1.58,0.02,...,3.46,4.45,4.58,1.44,2.09,1.99,1.74,1.26,245.34,1.25126
2015-02-22,17912.079,234.747,239.811,12066.7,5.7,5344.0,2097.45,2.11,1.58,0.02,...,3.46,4.45,4.58,1.44,2.09,1.99,1.74,1.26,245.99,1.25126
2015-02-23,17912.079,234.747,239.811,12066.7,5.7,5344.0,2110.30,2.13,1.61,0.02,...,3.46,4.45,4.53,1.46,2.11,2.01,1.75,1.26,236.83,1.25126
2015-02-24,17912.079,234.747,239.811,12066.7,5.7,5344.0,2109.66,2.06,1.56,0.01,...,3.46,4.45,4.56,1.42,2.04,1.95,1.72,1.26,239.49,1.25126


# Returns

In [225]:
# import datetime
# import yfinance as yf
# def get_price_matrix(tickers, start_date='1990-01-01', end_date=datetime.datetime.now().strftime('%Y-%m-%d')):
#     return yf.download(tickers, start_date, end_date)['Close']

# px = get_price_matrix(tickers = ['IVV', 'AGG', 'TLT',
#                             '^SP500-15',
#                             '^SP500-20', '^SP500-25', '^SP500-30', '^SP500-35', '^SP500-40', '^SP500-45', '^SP500-50', '^SP500-55', '^SP500-60',
#                             '^GSPE'
#                             ])

# px.rename(columns={'^SP500-15':"Materials",
#                    '^SP500-20':"Industrials", 
#                    '^SP500-25':"Consumer Discretionary", 
#                    '^SP500-30':"Consumer Staples", 
#                    '^SP500-35':"Health Care",
#                    '^SP500-40':"Financials",
#                    '^SP500-45':"Info Tech",
#                    '^SP500-50':"Comm Services",
#                    '^SP500-55':"Utilities",
#                    '^SP500-60':"Real Estate",
#                    '^GSPE':"Energy"
#                    }, inplace=True)

In [226]:
sprets = pd.read_excel(r"C:\Users\micha\OneDrive\Documents\data\spsectorpx.xlsx").set_index('Date').ffill().dropna()
sprets.index.name = 'date'
sprets.head()

,SPX,Info Tech,Financials,Consumer Discretionary,Health Care,Comm Services,Industrials,Consumer Staples,Energy,Utilities,Real Estate,Materials
date,,,,,,,,,,,,
2001-10-09,1056.75,282.14,323.74,205.8304,384.73,186.84,231.76,214.63,212.40,161.13,100.00,119.11
2001-10-10,1080.99,292.25,329.30,212.2388,391.11,188.55,237.77,217.03,219.86,163.84,99.19,122.13
2001-10-11,1097.43,308.68,333.50,219.5457,387.43,185.57,243.57,215.23,218.96,164.49,99.74,126.10
2001-10-12,1091.65,311.21,328.67,216.7271,388.68,183.25,241.63,212.67,219.07,163.94,99.48,125.35
2001-10-15,1089.98,307.85,331.49,216.6667,392.98,181.48,241.74,211.87,214.47,162.33,100.00,124.40


In [227]:
log_sprets = np.log(sprets / sprets.shift(1)).dropna()
log_sprets.tail()

,SPX,Info Tech,Financials,Consumer Discretionary,Health Care,Comm Services,Industrials,Consumer Staples,Energy,Utilities,Real Estate,Materials
date,,,,,,,,,,,,
2025-02-11,0.000340,0.001829,0.002630,-0.012355,-0.002197,-0.001338,0.000230,0.009048,0.007546,0.004924,0.005550,0.005355
2025-02-12,-0.002728,-0.000473,-0.004153,-0.002987,-0.001124,0.000382,-0.005840,0.002324,-0.027286,-0.001415,-0.009153,-0.006841
2025-02-13,0.010372,0.015127,0.007218,0.015873,0.003960,0.010756,0.000943,0.009628,0.007667,0.001440,0.008362,0.016955
2025-02-14,-0.000072,0.005997,0.001419,-0.002689,-0.011208,0.004098,-0.002866,-0.011640,0.001321,-0.005123,-0.004456,-0.003283
2025-02-18,0.002442,0.005522,0.007653,-0.005173,-0.002594,-0.012699,0.007687,0.001201,0.013578,0.009332,0.004041,0.012213


In [228]:
n_fwd = 252
fwd_log_sprets = log_sprets.rolling(window=n_fwd, min_periods=n_fwd).sum().shift(-n_fwd)
fwd_log_sprets.columns = [f'{col}_fwd_1y' for col in fwd_log_sprets.columns]
fwd_log_sprets.dropna(inplace=True)
fwd_log_sprets.head()

,SPX_fwd_1y,Info Tech_fwd_1y,Financials_fwd_1y,Consumer Discretionary_fwd_1y,Health Care_fwd_1y,Comm Services_fwd_1y,Industrials_fwd_1y,Consumer Staples_fwd_1y,Energy_fwd_1y,Utilities_fwd_1y,Real Estate_fwd_1y,Materials_fwd_1y
date,,,,,,,,,,,,
2001-10-10,-0.296133,-0.473163,-0.221020,-0.207547,-0.257040,-0.785714,-0.305261,-0.045483,-0.226321,-0.679568,-0.244697,-0.130856
2001-10-11,-0.272911,-0.466666,-0.184759,-0.193671,-0.234807,-0.714222,-0.282863,-0.021272,-0.203852,-0.669774,-0.218667,-0.117773
2001-10-12,-0.260331,-0.470859,-0.165313,-0.176557,-0.214670,-0.704397,-0.279945,0.004785,-0.185330,-0.676332,-0.233424,-0.116906
2001-10-15,-0.212551,-0.390489,-0.103188,-0.123943,-0.203564,-0.648954,-0.231722,0.018333,-0.131462,-0.658576,-0.203954,-0.059108
2001-10-16,-0.243871,-0.478126,-0.134903,-0.148893,-0.207074,-0.653216,-0.255899,-0.002415,-0.159664,-0.719638,-0.200479,-0.094323


# Earnings bf1

In [229]:
def merge_numbered_columns(df):
    """
    Merge numbered columns into a single DataFrame where every two columns
    represent a date-value pair.
    """
    series_list = []
    col_names = []
    
    for i in range(0, len(df.columns), 2):
        # Extract date and value columns
        date_col = df.iloc[:, i]
        value_col = df.iloc[:, i+1]
        
        temp_df = pd.DataFrame({
            'date': pd.to_datetime(date_col),
            f'series_{i//2}': value_col
        })
        
        temp_df.set_index('date', inplace=True)
        temp_df = temp_df.drop_duplicates(keep='last')
        
        series_list.append(temp_df)
        col_names.append(df.columns[i+1])
    
    result = pd.concat(series_list, axis=1)
    result.columns = col_names
    
    result = result.sort_index()
    
    return result


bf1 = pd.read_excel(r"C:\Users\micha\OneDrive\Documents\data\bf1eps.xlsx", skiprows=1).iloc[1:].dropna(axis=1, how='all').dropna(axis=0, how='all')
bf1 = merge_numbered_columns(bf1).ffill().fillna(1)

In [230]:
import warnings
warnings.filterwarnings('ignore')

def calculate_rsi(series, period=14):
    """Calculate RSI for a given series."""
    delta = series.pct_change()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def create_earnings_features(fwd_earnings_df):
    """
    Create features from forward earnings time series, preserving column names.
    
    Args:
        fwd_earnings_df (pd.DataFrame): DataFrame with forward earnings columns
    Returns:
        pd.DataFrame: Features with column-specific names
    """
    features = pd.DataFrame(index=fwd_earnings_df.index)
    
    # Create features for each earnings estimate column
    for col in fwd_earnings_df.columns:
        series = fwd_earnings_df[col]
        
        # Growth rates at different horizons
        features[f'{col}_1bf_eps_pct_chg_1m'] = series.pct_change(21)
        features[f'{col}_1bf_eps_pct_chg_3m'] = series.pct_change(63)
        features[f'{col}_1bf_eps_pct_chg_6m'] = series.pct_change(126)
        features[f'{col}_1bf_eps_pct_chg_12m'] = series.pct_change(252)
        
        # Z-scores
        features[f'{col}1bf_eps_zscore_3m'] = (
            series - series.rolling(63).mean()
        ) / series.rolling(63).std()
        
        # Trend strength
        features[f'{col}_trend_1m'] = series.pct_change(21).rolling(21).mean()

        # RSI crossovers and extremes
        features[f'{col}_1bf_eps_rsi_21d'] = calculate_rsi(series, period=21)    
        features[f'{col}_1bf_eps_rsi_63d'] = calculate_rsi(series, period=63)    
        features[f'{col}_1bf_eps_rsi_126d'] = calculate_rsi(series, period=126)    

        features[f'{col}_1bf_eps_rsi_cross'] = (
            features[f'{col}_1bf_eps_rsi_21d'] - features[f'{col}_1bf_eps_rsi_63d']
        )
        features[f'{col}_1bf_eps_rsi_extreme'] = (
            (features[f'{col}_1bf_eps_rsi_21d'] < 30) | (features[f'{col}_1bf_eps_rsi_21d'] > 70)
        ).astype(int)
        
    return features

In [231]:
bf1_features = create_earnings_features(bf1).dropna()

In [232]:
bf1_features.head()

,SPX Index_1bf_eps_pct_chg_1m,SPX Index_1bf_eps_pct_chg_3m,SPX Index_1bf_eps_pct_chg_6m,SPX Index_1bf_eps_pct_chg_12m,SPX Index1bf_eps_zscore_3m,SPX Index_trend_1m,SPX Index_1bf_eps_rsi_21d,SPX Index_1bf_eps_rsi_63d,SPX Index_1bf_eps_rsi_126d,SPX Index_1bf_eps_rsi_cross,...,S5RLST Index_1bf_eps_pct_chg_3m,S5RLST Index_1bf_eps_pct_chg_6m,S5RLST Index_1bf_eps_pct_chg_12m,S5RLST Index1bf_eps_zscore_3m,S5RLST Index_trend_1m,S5RLST Index_1bf_eps_rsi_21d,S5RLST Index_1bf_eps_rsi_63d,S5RLST Index_1bf_eps_rsi_126d,S5RLST Index_1bf_eps_rsi_cross,S5RLST Index_1bf_eps_rsi_extreme
date,,,,,,,,,,,,,,,,,,,,,
2016-09-19,0.008627,0.016801,0.035041,0.003758,2.354575,0.005039,74.494402,72.498176,75.466464,1.996226,...,4.8429,4.8429,4.8429,7.811266,0.230614,100.000000,100.000000,100.000000,0.0,1
2016-09-20,0.007717,0.016532,0.034340,0.004427,2.265029,0.005251,73.111870,72.300541,75.218123,0.811329,...,4.8052,4.8052,4.8052,5.456522,0.459433,99.866946,99.866946,99.866946,0.0,1
2016-09-21,0.007848,0.016923,0.034602,0.004845,2.278214,0.005492,73.320681,72.586774,75.311706,0.733907,...,4.8018,4.8018,4.8018,4.422120,0.688090,99.854886,99.854886,99.854886,0.0,1
2016-09-22,0.007972,0.016391,0.035434,0.005916,2.293022,0.005708,73.513939,72.195232,75.765682,1.318707,...,4.8018,4.8018,4.8018,3.800540,0.916748,99.854886,99.854886,99.854886,0.0,1
2016-09-23,0.007811,0.016182,0.036160,0.006528,2.178311,0.005952,73.219683,72.019948,76.558857,1.199735,...,4.7896,4.7896,4.7896,3.364470,1.144824,99.811610,99.811610,99.811610,0.0,1


# Merge

In [233]:
ff.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1990-01-03,0.0144,-0.0067,-0.0007,0.0019,-0.0044,0.00026,-1.09
1990-01-04,-0.0006,0.0072,-0.0026,0.0014,-0.0043,0.00026,-0.31
1990-01-05,-0.0071,0.0044,-0.0022,-0.0006,-0.0003,0.00026,-0.38
1990-01-08,-0.0085,0.0074,-0.0021,-0.0001,-0.0012,0.00026,-0.33
1990-01-09,0.0030,-0.0040,-0.0026,0.0006,0.0031,0.00026,0.18


In [234]:
macro.head()

,GDP,CPIAUCSL,CPILFESL,PCE,UNRATE,JTSJOL,SP500,DGS10,DGS5,DGS1MO,...,AAA,BAA,BAMLH0A0HYM2,T10Y2Y,T10Y3M,T10YFF,T10YIE,T5YFFM,CBBTCUSD,EXPINF2YR
date,,,,,,,,,,,,,,,,,,,,,
2015-02-20,17912.079,234.747,239.811,12066.7,5.7,5344.0,2097.45,2.11,1.58,0.02,...,3.46,4.45,4.58,1.44,2.09,1.99,1.74,1.26,240.90,1.25126
2015-02-21,17912.079,234.747,239.811,12066.7,5.7,5344.0,2097.45,2.11,1.58,0.02,...,3.46,4.45,4.58,1.44,2.09,1.99,1.74,1.26,245.34,1.25126
2015-02-22,17912.079,234.747,239.811,12066.7,5.7,5344.0,2097.45,2.11,1.58,0.02,...,3.46,4.45,4.58,1.44,2.09,1.99,1.74,1.26,245.99,1.25126
2015-02-23,17912.079,234.747,239.811,12066.7,5.7,5344.0,2110.30,2.13,1.61,0.02,...,3.46,4.45,4.53,1.46,2.11,2.01,1.75,1.26,236.83,1.25126
2015-02-24,17912.079,234.747,239.811,12066.7,5.7,5344.0,2109.66,2.06,1.56,0.01,...,3.46,4.45,4.56,1.42,2.04,1.95,1.72,1.26,239.49,1.25126


In [235]:
log_sprets.head()

,SPX,Info Tech,Financials,Consumer Discretionary,Health Care,Comm Services,Industrials,Consumer Staples,Energy,Utilities,Real Estate,Materials
date,,,,,,,,,,,,
2001-10-10,0.022679,0.035206,0.017028,0.030660,0.016447,0.009111,0.025601,0.011120,0.034520,0.016679,-0.008133,0.025039
2001-10-11,0.015094,0.054696,0.012674,0.033848,-0.009454,-0.015931,0.024101,-0.008328,-0.004102,0.003959,0.005530,0.031989
2001-10-12,-0.005281,0.008163,-0.014589,-0.012921,0.003221,-0.012581,-0.007997,-0.011966,0.000502,-0.003349,-0.002610,-0.005965
2001-10-15,-0.001531,-0.010855,0.008543,-0.000279,0.011002,-0.009706,0.000455,-0.003769,-0.021221,-0.009869,0.005214,-0.007608
2001-10-16,0.006912,0.018123,0.015090,0.005591,0.001043,-0.015437,0.000000,-0.001937,0.009144,0.020125,-0.016434,0.009122


In [236]:
fwd_log_sprets.head()

,SPX_fwd_1y,Info Tech_fwd_1y,Financials_fwd_1y,Consumer Discretionary_fwd_1y,Health Care_fwd_1y,Comm Services_fwd_1y,Industrials_fwd_1y,Consumer Staples_fwd_1y,Energy_fwd_1y,Utilities_fwd_1y,Real Estate_fwd_1y,Materials_fwd_1y
date,,,,,,,,,,,,
2001-10-10,-0.296133,-0.473163,-0.221020,-0.207547,-0.257040,-0.785714,-0.305261,-0.045483,-0.226321,-0.679568,-0.244697,-0.130856
2001-10-11,-0.272911,-0.466666,-0.184759,-0.193671,-0.234807,-0.714222,-0.282863,-0.021272,-0.203852,-0.669774,-0.218667,-0.117773
2001-10-12,-0.260331,-0.470859,-0.165313,-0.176557,-0.214670,-0.704397,-0.279945,0.004785,-0.185330,-0.676332,-0.233424,-0.116906
2001-10-15,-0.212551,-0.390489,-0.103188,-0.123943,-0.203564,-0.648954,-0.231722,0.018333,-0.131462,-0.658576,-0.203954,-0.059108
2001-10-16,-0.243871,-0.478126,-0.134903,-0.148893,-0.207074,-0.653216,-0.255899,-0.002415,-0.159664,-0.719638,-0.200479,-0.094323


In [237]:
bf1.head()

,SPX Index,S5COND Index,S5ENRS Index,S5HLTH Index,S5INFT Index,S5TELS Index,S5CONS Index,S5FINL Index,S5INDU Index,S5MATR Index,S5UTIL Index,S5RLST Index
date,,,,,,,,,,,,
2000-01-03,58.5285,9.2917,9.3551,12.5005,15.9120,13.2138,11.3596,21.9164,11.8646,8.6742,10.9234,1.0
2000-01-04,58.4263,9.2892,9.3551,12.5254,15.9240,13.2487,11.3672,21.9060,11.8517,8.6862,10.9331,1.0
2000-01-05,58.2950,9.2843,9.3409,12.5245,15.8030,13.1906,11.4001,21.9790,11.8439,8.7083,10.9215,1.0
2000-01-06,58.0814,9.2920,9.3135,12.5297,15.8085,12.9901,11.4180,21.9523,11.8333,8.6607,10.9128,1.0
2000-01-07,57.8483,9.2742,9.3837,12.5136,15.6651,12.9901,11.4183,21.9523,11.8333,8.6409,10.9128,1.0


In [238]:
bf1_features.head()

,SPX Index_1bf_eps_pct_chg_1m,SPX Index_1bf_eps_pct_chg_3m,SPX Index_1bf_eps_pct_chg_6m,SPX Index_1bf_eps_pct_chg_12m,SPX Index1bf_eps_zscore_3m,SPX Index_trend_1m,SPX Index_1bf_eps_rsi_21d,SPX Index_1bf_eps_rsi_63d,SPX Index_1bf_eps_rsi_126d,SPX Index_1bf_eps_rsi_cross,...,S5RLST Index_1bf_eps_pct_chg_3m,S5RLST Index_1bf_eps_pct_chg_6m,S5RLST Index_1bf_eps_pct_chg_12m,S5RLST Index1bf_eps_zscore_3m,S5RLST Index_trend_1m,S5RLST Index_1bf_eps_rsi_21d,S5RLST Index_1bf_eps_rsi_63d,S5RLST Index_1bf_eps_rsi_126d,S5RLST Index_1bf_eps_rsi_cross,S5RLST Index_1bf_eps_rsi_extreme
date,,,,,,,,,,,,,,,,,,,,,
2016-09-19,0.008627,0.016801,0.035041,0.003758,2.354575,0.005039,74.494402,72.498176,75.466464,1.996226,...,4.8429,4.8429,4.8429,7.811266,0.230614,100.000000,100.000000,100.000000,0.0,1
2016-09-20,0.007717,0.016532,0.034340,0.004427,2.265029,0.005251,73.111870,72.300541,75.218123,0.811329,...,4.8052,4.8052,4.8052,5.456522,0.459433,99.866946,99.866946,99.866946,0.0,1
2016-09-21,0.007848,0.016923,0.034602,0.004845,2.278214,0.005492,73.320681,72.586774,75.311706,0.733907,...,4.8018,4.8018,4.8018,4.422120,0.688090,99.854886,99.854886,99.854886,0.0,1
2016-09-22,0.007972,0.016391,0.035434,0.005916,2.293022,0.005708,73.513939,72.195232,75.765682,1.318707,...,4.8018,4.8018,4.8018,3.800540,0.916748,99.854886,99.854886,99.854886,0.0,1
2016-09-23,0.007811,0.016182,0.036160,0.006528,2.178311,0.005952,73.219683,72.019948,76.558857,1.199735,...,4.7896,4.7896,4.7896,3.364470,1.144824,99.811610,99.811610,99.811610,0.0,1


In [239]:
merge_res = log_sprets.merge(
    fwd_log_sprets, left_index=True, right_index=True, how='outer').merge(
    ff, left_index=True, right_index=True, how='outer').merge(
    macro, left_index=True, right_index=True, how='outer').merge(
    bf1, left_index=True, right_index=True, how='outer').merge(
    bf1_features, left_index=True, right_index=True, how='outer').dropna()
    

In [240]:
merge_res.shape

(1864, 207)

In [241]:
print(list(merge_res.columns))

['SPX', 'Info Tech', 'Financials', 'Consumer Discretionary', 'Health Care', 'Comm Services', 'Industrials', 'Consumer Staples', 'Energy', 'Utilities', 'Real Estate', 'Materials', 'SPX_fwd_1y', 'Info Tech_fwd_1y', 'Financials_fwd_1y', 'Consumer Discretionary_fwd_1y', 'Health Care_fwd_1y', 'Comm Services_fwd_1y', 'Industrials_fwd_1y', 'Consumer Staples_fwd_1y', 'Energy_fwd_1y', 'Utilities_fwd_1y', 'Real Estate_fwd_1y', 'Materials_fwd_1y', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   ', 'GDP', 'CPIAUCSL', 'CPILFESL', 'PCE', 'UNRATE', 'JTSJOL', 'SP500', 'DGS10', 'DGS5', 'DGS1MO', 'DGS3MO', 'DGS6MO', 'DGS1', 'DGS2', 'DGS3', 'DGS7', 'DGS20', 'DGS30', 'DCOILWTICO', 'DCOILBRENTEU', 'DAAA', 'DBAA', 'AAA', 'BAA', 'BAMLH0A0HYM2', 'T10Y2Y', 'T10Y3M', 'T10YFF', 'T10YIE', 'T5YFFM', 'CBBTCUSD', 'EXPINF2YR', 'SPX Index', 'S5COND Index', 'S5ENRS Index', 'S5HLTH Index', 'S5INFT Index', 'S5TELS Index', 'S5CONS Index', 'S5FINL Index', 'S5INDU Index', 'S5MATR Index', 'S5UTIL Index', 'S5RLST Index', '

In [242]:
merge_res.tail()

,SPX,Info Tech,Financials,Consumer Discretionary,Health Care,Comm Services,Industrials,Consumer Staples,Energy,Utilities,...,S5RLST Index_1bf_eps_pct_chg_3m,S5RLST Index_1bf_eps_pct_chg_6m,S5RLST Index_1bf_eps_pct_chg_12m,S5RLST Index1bf_eps_zscore_3m,S5RLST Index_trend_1m,S5RLST Index_1bf_eps_rsi_21d,S5RLST Index_1bf_eps_rsi_63d,S5RLST Index_1bf_eps_rsi_126d,S5RLST Index_1bf_eps_rsi_cross,S5RLST Index_1bf_eps_rsi_extreme
2024-02-08,0.000570,0.001428,-0.004948,0.002924,-0.001720,0.003888,0.000536,-0.000358,0.010818,-0.008309,...,0.004676,-0.015339,-0.003206,1.903787,0.005210,62.120802,55.102605,43.054825,7.018196,0.0
2024-02-09,0.005726,0.014888,0.003178,0.009839,-0.000554,0.007331,0.001162,-0.008550,-0.015745,0.004196,...,0.005200,-0.015886,-0.005205,1.794318,0.005370,59.989871,55.731867,42.776347,4.258003,0.0
2024-02-12,-0.000949,-0.007766,0.004151,-0.002763,0.000977,-0.001819,0.001232,0.006323,0.010451,0.011383,...,0.006694,-0.017246,-0.004716,2.050297,0.005493,60.106431,57.282370,42.046355,2.824061,0.0
2024-02-13,-0.013769,-0.015710,-0.012861,-0.019796,-0.008724,-0.013525,-0.010172,-0.009888,-0.011522,-0.017025,...,0.006588,-0.017545,-0.003615,1.969856,0.005448,57.744734,57.185172,41.883290,0.559562,0.0
2024-02-14,0.009534,0.010894,0.009573,0.010163,0.008045,0.014071,0.016579,-0.001930,-0.001733,0.005380,...,0.006410,-0.018750,-0.004086,1.778857,0.005508,60.239935,56.966380,41.267499,3.273555,0.0


In [243]:
merge_res = merge_res[~merge_res.index.duplicated(keep='first')]


In [244]:
merge_res.shape

(1864, 207)

In [ ]:
merge_res.to_csv(fp_out)

: 